### ネガポジ分析
感情値の表現方法には、以下の考え方がある。
- 肯定的か否定的かの2値
- 肯定的か否定的か中立かの3択
- 肯定的を+1、否定的を-1として（もしくは区間[M, -M]）その間の連続値

感情値辞書（日本語）
- [単語感情極性対応表](http://www.lr.pi.titech.ac.jp/~takamura/pndic_ja.html)： 約55000語に-1～+1の感情極性値が付けられている。
- [日本語評価極性辞書](http://www.cl.ecei.tohoku.ac.jp/index.php?Open%20Resources%2FJapanese%20Sentiment%20Polarity%20Dictionary)
- [Polar Phrase Dictionary](http://www.tkl.iis.u-tokyo.ac.jp/~kaji/polardic/)

感情値辞書（英語）
- AFINN-111: 2477個の語句が-4から4までの整数の感情値を振られている。  
[AFINNのダウンロードURL](http://www2.imm.dtu.dk/pubdb/views/edoc_download.php/6010/zip/imm6010.zip)
- SentiWordNet： 意味辞書であるWordNetに掲載されている概念単位に対して、感情値を割り振ろうとするもの。
- VADER： 辞書とルールの組み合わせで感情値を求める。7516語のエントリを持つ。  以下のコードで要ダウンロード。
>import nltk  
>nltk.download('vader_lexicon')
- NaiveBayesClassifier： 機械学習を使った感情値判定システム。教師有学習。学習には以下のリンク先へ。  
[映画の感想データ5331文のURL](http://www.cs.cornell.edu/people/pabo/movie-review-data/ "sentence polarity dataset v1.0")

In [1]:
# -*- coding: utf-8 -*-
# リスト 5-10 SentiWordNet による単語の感情値の和を求める
from nltk.tokenize import *

In [2]:
AFINNfile = 'AFINN/AFINN-111.txt'
sentiment_dictionary = {}

In [3]:
for line in open(AFINNfile):  # AFINN-111辞書の読み込み
    word, score = line.split('\t')
    sentiment_dictionary[word] = int(score)

In [4]:
sentiment_dictionary

{'abandon': -2,
 'abandoned': -2,
 'abandons': -2,
 'abducted': -2,
 'abduction': -2,
 'abductions': -2,
 'abhor': -3,
 'abhorred': -3,
 'abhorrent': -3,
 'abhors': -3,
 'abilities': 2,
 'ability': 2,
 'aboard': 1,
 'absentee': -1,
 'absentees': -1,
 'absolve': 2,
 'absolved': 2,
 'absolves': 2,
 'absolving': 2,
 'absorbed': 1,
 'abuse': -3,
 'abused': -3,
 'abuses': -3,
 'abusive': -3,
 'accept': 1,
 'accepted': 1,
 'accepting': 1,
 'accepts': 1,
 'accident': -2,
 'accidental': -2,
 'accidentally': -2,
 'accidents': -2,
 'accomplish': 2,
 'accomplished': 2,
 'accomplishes': 2,
 'accusation': -2,
 'accusations': -2,
 'accuse': -2,
 'accused': -2,
 'accuses': -2,
 'accusing': -2,
 'ache': -2,
 'achievable': 1,
 'aching': -2,
 'acquit': 2,
 'acquits': 2,
 'acquitted': 2,
 'acquitting': 2,
 'acrimonious': -3,
 'active': 1,
 'adequate': 1,
 'admire': 3,
 'admired': 3,
 'admires': 3,
 'admiring': 3,
 'admit': -1,
 'admits': -1,
 'admitted': -1,
 'admonish': -2,
 'admonished': -2,
 'adopt': 

In [5]:
# 分析対象
str = '''The first music is good, but the second and the third musics \
 are terrible and boring.  It is a bad idea to buy this CD.'''
for sent in sent_tokenize(str):
    words = word_tokenize(sent.lower())
    score = sum(sentiment_dictionary.get(word, 0) for word in words)
    print(score)

-3
-3


各センテンスの分析結果

The first music is good, but the second and the third musics are terrible and boring.

good +3

terrible -3

boring -3

It is a bad idea to buy this CD.

bad -3

In [6]:
# ポジネガスコアを別々に集計
result = []
for sent in sent_tokenize(str):
    print(sent)
    words = word_tokenize(sent.lower())
    pos = 0
    neg = 0
    for word in words:
        score = sentiment_dictionary.get(word, 0)
        if score > 0:
            pos += score
        if score < 0:
            neg += score
    result.append([pos, neg])
for u in result:
    print(u)

The first music is good, but the second and the third musics  are terrible and boring.
It is a bad idea to buy this CD.
[3, -6]
[0, -3]


感情値を単純に算出することができたが、以下のような懸念点について検討する必要がある。
- 感情値の和は、文章の長さ（文字量）に依存している。そのため、文の長さを正規化する、語数で割るなどの対策が必要。
- 文毎に感情値を求める、という考え方でよいかどうか。
- 否定修飾語も正の値としてカウントしてしまう。例：not goodなど。
- 増幅修飾がカウントされない。例：very goodなど。
- 辞書にない単語がカウントされない。

### NLTKのsentiment analysisパッケージの例
自然言語処理ライブラリであるNLTKの中に、英語を対象とした感情値分析ツールがある。

In [8]:
# VADER辞書の活用
# 結構楽に評価できる。顔文字も考慮されている
# 結果の見方{negative, neutral, positive, compound: VADERで定義する総合的な感情の評価値}
from nltk.sentiment.vader import SentimentIntensityAnalyzer
vader_analyzer = SentimentIntensityAnalyzer()
sent = 'I am happy'
result = vader_analyzer.polarity_scores(sent)
print(sent + '\n', result)
sent_with_face = 'I am happy :)' # 顔文字付き
result2 = vader_analyzer.polarity_scores(sent_with_face)
print(sent_with_face + '\n', result2)

I am happy
 {'neg': 0.0, 'neu': 0.213, 'pos': 0.787, 'compound': 0.5719}
I am happy :)
 {'neg': 0.0, 'neu': 0.13, 'pos': 0.87, 'compound': 0.7717}
